# Tensorflow Adversarial Embedding MNIST demo for Securing AI Lab deployment

This demo will cover the adversarial clean label backdoor attack on an MNIST-LeNet model.
The following two sections cover experiment setup and is similar across all demos.
To access demo results in MlFlow, please follow the general experiment setup steps outlined in `basic-mlflow-demo`.

## Setup: Experiment Name and MNIST Dataset

Here we will import the necessary Python modules and ensure the proper environment variables are set so that all the code blocks will work as expected.

**Important: Users will need to verify or update the following parameters:**

- Ensure that the `USERNAME` parameter is set to your own name.
- Ensure that the `DATASET_DIR` parameter is set to the location of the MNIST dataset directory. Currently set to `/nfs/data/Mnist` as the default location.
- (Optional) Set the `EXPERIMENT_NAME` parameter to your own preferred experiment name.

Other parameters can be modified to alter the RESTful API and MLFlow tracking addresses. 

In [1]:
# Import packages from the Python standard library
import os
import pprint
import time
import warnings
from pathlib import Path
from typing import Tuple

# Filter out warning messages
warnings.filterwarnings("ignore")

# Please enter custom username here.
USERNAME = "howard"

# Ensure that the dataset location is properly set here.
DATASET_DIR = "/nfs/data/Mnist"

# Experiment name (note the username_ prefix convention)
EXPERIMENT_NAME = f"{USERNAME}_mnist_clean_label_backdoor"

# Address for connecting the docker container to exposed ports on the host device
HOST_DOCKER_INTERNAL = "host.docker.internal"
# HOST_DOCKER_INTERNAL = "172.17.0.1"

# Testbed API ports
RESTAPI_PORT = "30080"
MLFLOW_TRACKING_PORT = "35000"

# Default address for accessing the RESTful API service
RESTAPI_ADDRESS = (
    f"http://{HOST_DOCKER_INTERNAL}:{RESTAPI_PORT}"
    if os.getenv("IS_JUPYTER_SERVICE")
    else f"http://localhost:{RESTAPI_PORT}"
)

# Override the AI_RESTAPI_URI variable, used to connect to RESTful API service
os.environ["AI_RESTAPI_URI"] = RESTAPI_ADDRESS

# Default address for accessing the MLFlow Tracking server
MLFLOW_TRACKING_URI = (
    f"http://{HOST_DOCKER_INTERNAL}:{MLFLOW_TRACKING_PORT}"
    if os.getenv("IS_JUPYTER_SERVICE")
    else f"http://localhost:{MLFLOW_TRACKING_PORT}"
)

# Override the MLFLOW_TRACKING_URI variable, used to connect to MLFlow Tracking service
os.environ["MLFLOW_TRACKING_URI"] = MLFLOW_TRACKING_URI

# Base API address
RESTAPI_API_BASE = f"{RESTAPI_ADDRESS}/api"

# Path to workflows archive
WORKFLOWS_TAR_GZ = Path("workflows.tar.gz")

# Import third-party Python packages
import numpy as np
import requests
from mlflow.tracking import MlflowClient

# Import utils.py file
import utils

# Create random number generator
rng = np.random.default_rng(54399264723942495723666216079516778448)

## Submit and run jobs

The entrypoints that we will be running in this example are implemented in the Python source files under `src/` and the `MLproject` file.
To run these entrypoints within the testbed architecture, we need to package those files up into an archive and submit it to the Testbed RESTful API to create a new job.
For convenience, the `Makefile` provides a rule for creating the archive file for this example, just run `make workflows`,

In [2]:
%%bash

# Create the workflows.tar.gz file
make workflows

tar czf workflows.tar.gz src/gaussian_augmentation.py src/infer.py src/defenses_image_preprocessing.py src/tasks.py src/attacks_poison_updated.py src/deploy_poison_data.py src/tracking_mlflow_updated.py src/gen_poison_data.py src/train.py src/import_keras.py src/spatial_smoothing.py src/jpeg_compression.py src/init_model.py src/gen_poison_clean_data.py src/gen_poison_model.py src/registry_art_updated.py src/estimators_keras_classifiers_updated.py src/data_tensorflow_updated.py MLproject
chmod 644 workflows.tar.gz


/home/hhuang/.conda/envs/tensorflow-mnist-classifier/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


To connect with the endpoint, we will use a client class defined in the `utils.py` file that is able to connect with the Testbed RESTful API using the HTTP protocol.
We connect using the client below, which uses the environment variable `AI_RESTAPI_URI` to figure out how to connect to the Testbed RESTful API,

In [3]:
restapi_client = utils.SecuringAIClient()

/home/hhuang/.conda/envs/tensorflow-mnist-classifier/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


We need to register an experiment under which to collect our job runs.
The code below checks if the relevant experiment exists.
If it does, then it just returns info about the experiment, if it doesn't, it then registers the new experiment.

In [4]:
response_experiment = restapi_client.get_experiment_by_name(name=EXPERIMENT_NAME)

if response_experiment is None or "Not Found" in response_experiment.get("message", []):
    response_experiment = restapi_client.register_experiment(name=EXPERIMENT_NAME)

response_experiment

{'experimentId': 22,
 'name': 'howard_mnist_clean_label_backdoor',
 'lastModified': '2021-01-25T20:03:56.964324',
 'createdOn': '2021-01-25T20:03:56.964324'}

We should also check which queues are available for running our jobs to make sure that the resources that we need are available.
The code below queries the Testbed API and returns a list of active queues.

In [5]:
restapi_client.list_queues()

[{'name': 'tensorflow_cpu',
  'lastModified': '2020-11-20T17:46:06.756687',
  'createdOn': '2020-11-20T17:46:06.756687',
  'queueId': 1},
 {'name': 'tensorflow_gpu',
  'lastModified': '2020-11-20T18:00:40.876888',
  'createdOn': '2020-11-20T18:00:40.876888',
  'queueId': 2},
 {'name': 'pytorch_cpu',
  'lastModified': '2020-11-20T19:52:36.079781',
  'createdOn': '2020-11-20T19:52:36.079781',
  'queueId': 5},
 {'name': 'pytorch_gpu',
  'lastModified': '2020-11-20T19:52:43.348460',
  'createdOn': '2020-11-20T19:52:43.348460',
  'queueId': 7}]

The following helper functions will recheck the job responses until the job is completed or a run ID is available. 
The run ID is needed to link dependencies between jobs.

In [6]:
def mlflow_run_id_is_not_known(job_response):
    return job_response["mlflowRunId"] is None and job_response["status"] not in [
        "failed",
        "finished",
    ]


def get_run_id(job_response):
    while mlflow_run_id_is_not_known(job_response):
        time.sleep(1)
        job_response = restapi_client.get_job_by_id(job_response["jobId"])
        
    return job_response


def wait_until_finished(job_response):
    # First make sure job has started.
    job_response = get_run_id(job_response)
    
    # Next re-check job until it has stopped running.
    while (job_response["status"] not in ["failed", "finished"]):
        time.sleep(1)
        job_response = restapi_client.get_job_by_id(job_response["jobId"])
    
    return job_response


# Helper function for viewing MLflow results
def get_mlflow_results(job_response):
    mlflow_client = MlflowClient()
    job_response = wait_until_finished(job_response)
    
    if(job_response['status']=="failed"):
        return {}
    
    run = mlflow_client.get_run(job_response["mlflowRunId"])  
    
    while(len(run.data.metrics) == 0):
        time.sleep(1)
        run = mlflow_client.get_run(job_response["mlflowRunId"])
        
    return run


def print_mlflow_results(response):
    results = get_mlflow_results(response)
    pprint.pprint(results.data.metrics)

## MNIST Training: Baseline Model

Next, we need to train our baseline model that will serve as a reference point for the effectiveness of our attacks.
We will be submitting our job to the `"tensorflow_gpu"` queue.

In [7]:
response_le_net_train = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="train",
    entry_point_kwargs=" ".join([
        "-P batch_size=256",
        f"-P register_model_name={EXPERIMENT_NAME}_le_net",
        "-P model_architecture=le_net",
        "-P epochs=30",
        f"-P data_dir_training={DATASET_DIR}/training",
        f"-P data_dir_testing={DATASET_DIR}/testing",
    ]),
    queue="tensorflow_gpu",
    timeout="1h",    
)

print("Training job for LeNet-5 neural network submitted")
print("")
pprint.pprint(response_le_net_train)

response_le_net_train = get_run_id(response_le_net_train)
print_mlflow_results(response_le_net_train)

Training job for LeNet-5 neural network submitted

{'createdOn': '2021-06-02T08:07:28.751618',
 'dependsOn': None,
 'entryPoint': 'train',
 'entryPointKwargs': '-P batch_size=256 -P '
                     'register_model_name=howard_mnist_clean_label_backdoor_le_net '
                     '-P model_architecture=le_net -P epochs=30 -P '
                     'data_dir_training=/nfs/data/Mnist/training -P '
                     'data_dir_testing=/nfs/data/Mnist/testing',
 'experimentId': 22,
 'jobId': '1634741f-f0cb-4e21-9afe-53dc4e1e898b',
 'lastModified': '2021-06-02T08:07:28.751618',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '1h',
 'workflowUri': 's3://workflow/c55c6252451140b9a353fd36edf6c12c/workflows.tar.gz'}
{'accuracy': 0.9768561124801636,
 'auc': 0.9988279342651367,
 'loss': 0.0771464533466278,
 'precision': 0.9808854460716248,
 'recall': 0.9738563299179077,
 'restored_epoch': 2.0,
 'stopped_epoch': 7.0,
 'training_time_in_minutes': 5.0481032,
 'val_ac

In [29]:
# Train a special model for making poisons
response_le_net_train_robust = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="train_madry_pgd",
    entry_point_kwargs=" ".join([
        "-P batch_size=256",
        f"-P register_model_name={EXPERIMENT_NAME}_robust_le_net",
        "-P model_architecture=le_net",
        "-P epochs=10",
        f"-P data_dir_training={DATASET_DIR}/training",
        f"-P data_dir_testing={DATASET_DIR}/testing",
    ]),
    queue="tensorflow_gpu",
    timeout="1h",
    
)

print("Training job for LeNet-5 neural network submitted")
print("")
pprint.pprint(response_le_net_train_robust)

response_le_net_train_robust = get_run_id(response_le_net_train_robust)
print_mlflow_results(response_le_net_train_robust)

/home/hhuang/.conda/envs/tensorflow-mnist-classifier/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Training job for LeNet-5 neural network submitted

{'createdOn': '2021-06-02T09:38:44.367335',
 'dependsOn': None,
 'entryPoint': 'train_madry_pgd',
 'entryPointKwargs': '-P batch_size=256 -P '
                     'register_model_name=howard_mnist_clean_label_backdoor_robust_le_net '
                     '-P model_architecture=le_net -P epochs=30 -P '
                     'data_dir_training=/nfs/data/Mnist/training -P '
                     'data_dir_testing=/nfs/data/Mnist/testing',
 'experimentId': 22,
 'jobId': '85268ee2-d315-4cf5-98f1-9e78a6c35d9e',
 'lastModified': '2021-06-02T09:38:44.367335',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '1h',
 'workflowUri': 's3://workflow/3f1e1c60d49d47fcbcde4ee1e7f84ed0/workflows.tar.gz'}


KeyboardInterrupt: 

### Generating Poisoned Images

Now we will create our set of poisoned images.
Start by submitting the poison generation job below.

In [8]:
## Create poisoned test images.
response_gen_poison_le_net_test = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="gen_poison_data",
    entry_point_kwargs=" ".join(
        [
            f"-P data_dir={DATASET_DIR}/testing",
            "-P batch_size=100",
            "-P target_class=1",
            "-P poison_fraction=1",
            "-P label_type=test"
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_le_net_train["jobId"],
)

print("Backdoor poison attack (LeNet-5 architecture) job submitted")
print("")
pprint.pprint(response_gen_poison_le_net_test)
print("")

response_gen_poison_le_net_test = get_run_id(response_gen_poison_le_net_test)

Backdoor poison attack (LeNet-5 architecture) job submitted

{'createdOn': '2021-06-02T08:12:55.075049',
 'dependsOn': '1634741f-f0cb-4e21-9afe-53dc4e1e898b',
 'entryPoint': 'gen_poison_data',
 'entryPointKwargs': '-P data_dir=/nfs/data/Mnist/testing -P batch_size=100 -P '
                     'target_class=1 -P poison_fraction=1 -P label_type=test',
 'experimentId': 22,
 'jobId': 'ee35f03e-21c9-459e-ab7a-6022b182526b',
 'lastModified': '2021-06-02T08:12:55.075049',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/6a58bbf3966a4a95b60e5569d08d69d4/workflows.tar.gz'}



In [33]:
## Create poisoned training images (clean_label)
response_gen_poison_le_net_train_clean = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="gen_poison_clean_data",
    entry_point_kwargs=" ".join(
        [
             f"-P model_name={EXPERIMENT_NAME}_robust_le_net",
            "-P model_version=none",
            f"-P data_dir={DATASET_DIR}/testing",
            "-P batch_size=200",
            "-P target_index=1",
            "-P poison_fraction=0.33",
            "-P label_type=train"
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_le_net_train["jobId"],
)

print("Backdoor poison attack (LeNet-5 architecture) job submitted")
print("")
pprint.pprint(response_gen_poison_le_net_train_clean)
print("")

response_gen_poison_le_net_train_clean = get_run_id(response_gen_poison_le_net_train_clean)

Backdoor poison attack (LeNet-5 architecture) job submitted

{'createdOn': '2021-06-02T10:03:42.877898',
 'dependsOn': '1634741f-f0cb-4e21-9afe-53dc4e1e898b',
 'entryPoint': 'gen_poison_clean_data',
 'entryPointKwargs': '-P '
                     'model_name=howard_mnist_clean_label_backdoor_robust_le_net '
                     '-P model_version=none -P '
                     'data_dir=/nfs/data/Mnist/testing -P batch_size=200 -P '
                     'target_index=1 -P poison_fraction=0.33 -P '
                     'label_type=train',
 'experimentId': 22,
 'jobId': '50ba53c6-4f20-4522-89fb-069f0e101b75',
 'lastModified': '2021-06-02T10:03:42.877898',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/2d706ed35a754d12a97ebc054b4a3341/workflows.tar.gz'}



## MNIST Training: Poisoned Model using a Clean Label technique

Next we will train our poisoned model using a clean label technique. 

In [32]:
# Now train a new model on the poisoned clean label images
response_le_net_train_backdoor_model_clean = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="train",
    entry_point_kwargs=" ".join([
        "-P batch_size=256",
        f"-P register_model_name={EXPERIMENT_NAME}_data_poison_le_net",
        "-P model_architecture=le_net",
        "-P epochs=30",
        f"-P data_dir_training={DATASET_DIR}/training",
        f"-P data_dir_testing={DATASET_DIR}/testing",
        "-P load_dataset_from_mlruns=true",
        f"-P dataset_run_id_training={response_gen_poison_le_net_train_clean['mlflowRunId']}",
        "-P adv_tar_name=adversarial_poison.tar.gz",
        "-P adv_data_dir=adv_poison_data"
    ]),
    depends_on=response_gen_poison_le_net_train_clean["jobId"],
    queue="tensorflow_gpu",
    timeout="1h",
    
)

print("Training job for LeNet-5 neural network submitted")
print("")
pprint.pprint(response_le_net_train_backdoor_model_clean)

response_le_net_train_backdoor_model_clean = get_run_id(response_le_net_train_backdoor_model_clean)
print_mlflow_results(response_le_net_train_backdoor_model_clean)

Training job for LeNet-5 neural network submitted

{'createdOn': '2021-06-02T09:57:37.847602',
 'dependsOn': '90bdf6c5-7997-4566-8589-9da6bfcee096',
 'entryPoint': 'train',
 'entryPointKwargs': '-P batch_size=256 -P '
                     'register_model_name=howard_mnist_clean_label_backdoor_data_poison_le_net '
                     '-P model_architecture=le_net -P epochs=30 -P '
                     'data_dir_training=/nfs/data/Mnist/training -P '
                     'data_dir_testing=/nfs/data/Mnist/testing -P '
                     'load_dataset_from_mlruns=true -P '
                     'dataset_run_id_training=3b6a77c80d174ac6bb459f9e9fe6400d '
                     '-P adv_tar_name=adversarial_poison.tar.gz -P '
                     'adv_data_dir=adv_poison_data',
 'experimentId': 22,
 'jobId': 'a2732b3b-11e7-43b3-817a-b12c804043cf',
 'lastModified': '2021-06-02T09:57:37.847602',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '1h',
 'workflowUri': 's3://wo

KeyboardInterrupt: 

## Model Evaluation: Poisoned vs Regular Models on Backdoor-Poisoned Images.

Below we will compare the results of the regular model vs poisoned-backdoor model on backdoor test images.

In [19]:
# Inference: Model trained on poisoned backdoor attack
response_infer_pos_model_clean = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_gen_poison_le_net_test['mlflowRunId']}",
            f"-P model_name={EXPERIMENT_NAME}_data_poison_le_net",
            "-P model_version=none",
            "-P batch_size=512",
            "-P adv_tar_name=adversarial_poison.tar.gz",
            "-P adv_data_dir=adv_poison_data",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_le_net_train_backdoor_model_clean["jobId"],
)

print("Inference job for LeNet-5 neural network submitted")
print("")

pprint.pprint(response_infer_pos_model_clean)
response_infer_pos_model_clean = get_run_id(response_infer_pos_model_clean)
print_mlflow_results(response_infer_pos_model_clean)

Inference job for LeNet-5 neural network submitted

{'createdOn': '2021-06-02T08:37:31.766959',
 'dependsOn': 'fc134c7e-64c4-4af0-878a-ae32283000c4',
 'entryPoint': 'infer',
 'entryPointKwargs': '-P run_id=360e53d8a70344bc8ab908debd1e437f -P '
                     'model_name=howard_mnist_clean_label_backdoor_data_poison_le_net '
                     '-P model_version=none -P batch_size=512 -P '
                     'adv_tar_name=adversarial_poison.tar.gz -P '
                     'adv_data_dir=adv_poison_data',
 'experimentId': 22,
 'jobId': '20c090e7-7b21-41ee-91d3-3924c5ab415f',
 'lastModified': '2021-06-02T08:37:31.766959',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/3e1da1945685477298efc765007fdea4/workflows.tar.gz'}
{'accuracy': 0.14790000021457672,
 'auc': 0.6006970405578613,
 'loss': 8.332611465454102,
 'precision': 0.14790000021457672,
 'recall': 0.14790000021457672}


In [20]:
# Inference: Regular model on poisoned test images
response_infer_reg_model = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_gen_poison_le_net_test['mlflowRunId']}",
            f"-P model_name={EXPERIMENT_NAME}_le_net",
            "-P model_version=none",
            "-P batch_size=512",
            "-P adv_tar_name=adversarial_poison.tar.gz",
            "-P adv_data_dir=adv_poison_data",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_le_net_train["jobId"],
)

print("Inference job for LeNet-5 neural network submitted")
print("")
pprint.pprint(response_infer_reg_model)
print_mlflow_results(response_infer_reg_model)

Inference job for LeNet-5 neural network submitted

{'createdOn': '2021-06-02T08:38:03.109778',
 'dependsOn': '1634741f-f0cb-4e21-9afe-53dc4e1e898b',
 'entryPoint': 'infer',
 'entryPointKwargs': '-P run_id=360e53d8a70344bc8ab908debd1e437f -P '
                     'model_name=howard_mnist_clean_label_backdoor_le_net -P '
                     'model_version=none -P batch_size=512 -P '
                     'adv_tar_name=adversarial_poison.tar.gz -P '
                     'adv_data_dir=adv_poison_data',
 'experimentId': 22,
 'jobId': '0c4ce22c-e530-4834-aee2-c706937a39d2',
 'lastModified': '2021-06-02T08:38:03.109778',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/227eb85ad9ad45828d1c3ea32abb026a/workflows.tar.gz'}
{'accuracy': 0.9873999953269958,
 'auc': 0.9996170401573181,
 'loss': 0.03629831066355109,
 'precision': 0.9892570376396179,
 'recall': 0.9853000044822693}


## Defending against the clean label poisoning attack

Now we will explore available defenses on the adversarial backdoor poisoning attack.
The following three jobs will run a selected defense (spatial smoothing, gaussian augmentation, or jpeg compression) and evaluate the defense on the baseline and backdoor trained models.

- The first job uses the selected defense entrypoint to apply a preprocessing defense over the poisoned test images.
- The second job runs the defended images against the poisoned backdoor model.
- The final job runs the defended images against the baseline model.

Ideally the defense will not impact the baseline model accuracy, while improving the backdoor model accuracy scores.

In [22]:
defenses = ["gaussian_augmentation", "spatial_smoothing", "jpeg_compression"]
defense = defenses[2]
response_poison_def = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point=defense,
    entry_point_kwargs=" ".join(
        [
            f"-P data_dir={DATASET_DIR}/testing",
            "-P batch_size=20",
            "-P load_dataset_from_mlruns=true",
            f"-P dataset_run_id={response_gen_poison_le_net_test['mlflowRunId']}",
            "-P dataset_tar_name=adversarial_poison.tar.gz",
            "-P dataset_name=adv_poison_data",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_gen_poison_le_net_test["jobId"],
)


print(f"FGM {defense} defense (LeNet architecture) job submitted")
print("")
pprint.pprint(response_poison_def)
print("")

response_poison_def = get_run_id(response_poison_def)

FGM jpeg_compression defense (LeNet architecture) job submitted

{'createdOn': '2021-06-02T08:40:45.548431',
 'dependsOn': 'ee35f03e-21c9-459e-ab7a-6022b182526b',
 'entryPoint': 'jpeg_compression',
 'entryPointKwargs': '-P data_dir=/nfs/data/Mnist/testing -P batch_size=20 -P '
                     'load_dataset_from_mlruns=true -P '
                     'dataset_run_id=360e53d8a70344bc8ab908debd1e437f -P '
                     'dataset_tar_name=adversarial_poison.tar.gz -P '
                     'dataset_name=adv_poison_data',
 'experimentId': 22,
 'jobId': '74db6834-92e1-488f-af29-01eefa21c2fb',
 'lastModified': '2021-06-02T08:40:45.548431',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/3c4bec354eb94bc09ace77d182a37d5e/workflows.tar.gz'}



In [23]:
# Inference: Poisoned model on poisoned test images.
response_infer_pos_model = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_poison_def['mlflowRunId']}",
            f"-P model_name={EXPERIMENT_NAME}_data_poison_le_net",
            f"-P model_version=none",
            "-P batch_size=512",
            f"-P adv_tar_name={defense}_dataset.tar.gz",
            "-P adv_data_dir=adv_testing",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_poison_def["jobId"],
)

print("Inference job for LeNet-5 neural network submitted")
print("")
pprint.pprint(response_infer_pos_model)
print_mlflow_results(response_infer_pos_model)


Inference job for LeNet-5 neural network submitted

{'createdOn': '2021-06-02T08:41:04.206798',
 'dependsOn': '74db6834-92e1-488f-af29-01eefa21c2fb',
 'entryPoint': 'infer',
 'entryPointKwargs': '-P run_id=c681461be82349c78eac0397e1fda7dd -P '
                     'model_name=howard_mnist_clean_label_backdoor_data_poison_le_net '
                     '-P model_version=none -P batch_size=512 -P '
                     'adv_tar_name=jpeg_compression_dataset.tar.gz -P '
                     'adv_data_dir=adv_testing',
 'experimentId': 22,
 'jobId': 'bce00cd1-ffcc-4342-ac66-21e8ac92ea21',
 'lastModified': '2021-06-02T08:41:04.206798',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/5eaaff02375c44c1ac1e1d1b7cbd1526/workflows.tar.gz'}
{'accuracy': 0.13840000331401825,
 'auc': 0.5923616886138916,
 'loss': 8.432003355026245,
 'precision': 0.13831382989883423,
 'recall': 0.13830000162124634}


In [24]:
# Inference: Regular model on poisoned test images.
response_infer_reg_model = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_poison_def['mlflowRunId']}",
            f"-P model_name={EXPERIMENT_NAME}_le_net",
            f"-P model_version=none",
            "-P batch_size=512",
            f"-P adv_tar_name={defense}_dataset.tar.gz",
            "-P adv_data_dir=adv_testing",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_poison_def["jobId"],
)

print("Inference job for LeNet-5 neural network submitted")
print("")
pprint.pprint(response_infer_reg_model)
print_mlflow_results(response_infer_reg_model)

Inference job for LeNet-5 neural network submitted

{'createdOn': '2021-06-02T08:41:46.861085',
 'dependsOn': '74db6834-92e1-488f-af29-01eefa21c2fb',
 'entryPoint': 'infer',
 'entryPointKwargs': '-P run_id=c681461be82349c78eac0397e1fda7dd -P '
                     'model_name=howard_mnist_clean_label_backdoor_le_net -P '
                     'model_version=none -P batch_size=512 -P '
                     'adv_tar_name=jpeg_compression_dataset.tar.gz -P '
                     'adv_data_dir=adv_testing',
 'experimentId': 22,
 'jobId': '7576bbe6-dd41-4ee9-8af2-b8e3c5eca99c',
 'lastModified': '2021-06-02T08:41:46.861085',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/4ab7916cd7ab47f081529168a763903a/workflows.tar.gz'}
{'accuracy': 0.9869999885559082,
 'auc': 0.9997146725654602,
 'loss': 0.03738134503364563,
 'precision': 0.989255964756012,
 'recall': 0.9851999878883362}


<a id='querying_cell'></a>
## Querying the MLFlow Tracking Service

Currently the lab API can only be used to register experiments and start jobs, so if users wish to extract their results programmatically, they can use the `MlflowClient()` class from the `mlflow` Python package to connect and query their results.
Since we captured the run ids generated by MLFlow, we can easily retrieve the data logged about one of our jobs and inspect the results.
To start the client, we simply need to run,

In [ ]:
mlflow_client = MlflowClient()

The client uses the environment variable `MLFLOW_TRACKING_URI` to figure out how to connect to the MLFlow Tracking Service, which we configured near the top of this notebook.
To query the results of one of our runs, we just need to pass the run id to the client's `get_run()` method.
As an example, let's query the run results for the patch attack applied to the LeNet-5 architecture,

In [ ]:
run_le_net = mlflow_client.get_run(response_le_net_train["mlflowRunId"])

If the request completed successfully, we should now be able to query data collected during the run.
For example, to review the collected metrics, we just use,

In [ ]:
pprint.pprint(run_le_net.data.metrics)

To review the run's parameters, we use,

In [ ]:
pprint.pprint(run_le_net.data.params)

To review the run's tags, we use,

In [ ]:
pprint.pprint(run_le_net.data.tags)

There are many things you can query using the MLFlow client.
[The MLFlow documentation gives a full overview of the methods that are available](https://www.mlflow.org/docs/latest/python_api/mlflow.tracking.html#mlflow.tracking.MlflowClient).